In [1]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
from time import sleep
import quandl 
import os
quandl.ApiConfig.api_key = os.getenv("QUANDL_API_KEY")
%matplotlib inline

In [2]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [3]:
# Get all Assets
assets = api.list_assets()

# Keep only tradeable assets
tradeable = [asset for asset in assets if asset.tradable ]

In [4]:
# Create a new empty DataFrame
asset_info_df = pd.DataFrame()
asset_info_df['symbol'] = pd.Series([asset.symbol for asset in assets if asset.exchange=="NASDAQ" or asset.exchange=="NYSE"])

# Display the first 10 asset tickers
display(asset_info_df.head(10))

,symbol
0,WRLSR
1,SODA_DELISTED
2,AESE
3,BBAR
4,NET
5,AFGD
6,EGC
7,AFGH
8,CO
9,AFI


In [5]:
print(asset_info_df.count())

symbol    7122
dtype: int64


In [6]:
print(7122/180)

39.56666666666667


In [7]:
asset_info = [asset.symbol for asset in assets]

In [ ]:
# Set timeframe to '1D'
timeframe = '1D'
start_date = pd.Timestamp("2018-06-18", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-06-23", tz="America/New_York").isoformat()

stockprices = []

i = 0
for asset in asset_info:
    print(asset)
    df = api.get_barset(
        asset,
        timeframe,
        limit=None,
        start=start_date,
        end=end_date,
        after=None,
        until=None,
    ).df


    
    dfq = quandl.get_table('SHARADAR/DAILY', ticker=asset)
    #print(df)
    if df.empty:
        print("     - Empty - Skip")
    elif dfq.empty:
        print("     - Empty - Skip")
    else:
        # format alpaca df for joining
        df = df.stack(level=0)
        df = df.rename_axis(('date', 'ticker'))
        df = df.reset_index()
        df['date'] = df['date'].dt.date
        df = df.set_index(["date", "ticker"])
        
        dfq = dfq.set_index(["date", "ticker"])
        dfb = df.join(dfq)
        print(dfb.head(2))
        #stockprices.append({"Ticker" : asset, "Metrics": dfb})
        stockprices.append(dfb)
        # print("Length:", len(stockprices))
        i += 1
        if i%3 == 0: 
            sleep(1)
        
        # take these two lines out when you're ready to run for all assets
        #if i > 10:
         #   break

WRLSR
     - Empty - Skip
SODA_DELISTED
     - Empty - Skip
AESE
                   close  high   low  open  volume lastupdated    ev  evebit  \
date       ticker                                                              
2019-08-12 AESE     4.10  5.10  3.76  4.20  357705  2019-08-12  35.3    18.3   
2019-08-13 AESE     3.57  4.08  3.42  4.01  249365  2019-08-13  30.9    16.0   

                   evebitda  marketcap   pb    pe  ps  
date       ticker                                      
2019-08-12 AESE        18.3       34.6  6.9  28.6 NaN  
2019-08-13 AESE        16.0       30.2  6.0  25.0 NaN  
BBAR
     - Empty - Skip
NET
                   close   high   low  open    volume lastupdated      ev  \
date       ticker                                                           
2019-09-13 NET     18.06  19.53  17.5  18.0  40464860  2019-09-16  5242.0   
2019-09-16 NET     18.67  19.55  18.3  18.6   9353971  2019-09-16  5426.8   

                   evebit  evebitda  marketcap    pb

In [ ]:
pd.concat(stockprices)

In [ ]:
# df = pd.DataFrame({'Index': [] })

In [ ]:
# pd.concat([df, dfq], join='inner', ignore_index=False)

In [ ]:
print(api.data_get(path, data=None))

In [ ]:
print(len(stockprices))


In [ ]:
stockprices.head()


In [ ]:
stockprices.to_csv("fullprocess.csv")